🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴

##Set of Fours \ Quartets card game

🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴🎴

**Liad Alkobi \ Jan 2022 \ NAYA College**

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT45jtDBDW1K6s_CYlQzpjzYjlSZHw6bq5qtA&usqp=CAU">

##Game Rules and Description:
---

[Hebrew wikipedia link](https://he.wikipedia.org/wiki/%D7%9E%D7%A9%D7%97%D7%A7_%D7%A8%D7%91%D7%99%D7%A2%D7%99%D7%95%D7%AA)

[English wikipedia link](https://en.wikipedia.org/wiki/Quartets_(card_game))

The objective in this game is to first achieve the number of sets defined in game intial parameters('sets_to_win').

Every player is dealt 4 cards in the beginning of the game.

Then every player in his turn ask for another player for a specific card , if the player guessed right he takes the card to himself and play another turn.

After creating a set ( 4 cards with the same subject) , he put the set cards a side.


# **Data and Intial Parameters**

In [104]:
import random as rnd
import copy
import itertools as it

In [105]:
#get data \ edit dict
deck = {'subj1' : ['subj_x1' , 'subjx2' , 'subjx3' , 'subjx4'] ,
        'subj2': ['subjy1' , 'subjy2' , 'subjy3' , 'subjy4'] , 
        'subj3': ['subj_t1' , 'subj_t2' , 'subj_t3' , 'subj_t4'] ,
        'subj4': ['subj_s1' , 'subj_s2' , 'subj_s3' , 'subj_s4'] }
dict_ = copy.deepcopy(deck)
#Parameters
sets_to_win = 1
amount_of_players = 3
min_cards_in_hand = 4
guess = True
deck_weight = 1.25 #the min ratio of : number of cards in deck / cards dealt 
#, which you wish would be (1 --> cards in deck >= cards dealt in order to the game to begin)


In [106]:
#check that there is enough cards / valid parameters
number_of_cards_in_deck = sum(list(map(lambda x: len(x) , deck.values() ) ) ) # total number of cards
#A - min amount of cards needed for having sets_to_win --> sets_to_win * 4
cards_for_sets = 4 * sets_to_win
#B - the avarage number of cards in players hand after the *all* deck dealt -->  number_of_cards_in_deck / amount_of_players
avg_cards_in_hand = number_of_cards_in_deck / amount_of_players
#C (amount_of_players * 4 ) * deck_weight --> the min amount of cards needed for the game to begin 
###C >= number of cards in deck !!**and**!! A >= B
if (amount_of_players * 4 ) * deck_weight >= number_of_cards_in_deck or cards_for_sets >= avg_cards_in_hand: #game do not begin
    end_game = True
    print(f'''to many players or not enough cards ; number of cards :{number_of_cards_in_deck} 
            , number of players: {amount_of_players} , number of sets to win : {sets_to_win} ''')
else:
    end_game = False
    print(f'game is on !!!')

game is on !!!


# **Player Class**

In [107]:
class player:
    def __init__(self , player_type = 'human' , cards = {} , sets = 0, amount_of_cards = 0):
        self.player_type = player_type
        self.cards = {}
        self.sets = sets
        self.amount_of_cards = amount_of_cards
    
    def __str__(self):
        return f'(Cards : {self.cards} , sets: {self.sets} , amount of cards in hand : {self.amount_of_cards} )'
    
    def add_card_from_deck(self , deck):
        if len(deck) != 0:
            subject_list = list(deck.keys())
            random_subject = rnd.choice(subject_list) #choose random subject\theme
            random_card = rnd.choice(deck[random_subject]) #Sub-theme
            #add card to player cards and remove from deck
            if random_subject in self.cards:
                self.cards[random_subject].append(random_card)
            else:
                self.cards[random_subject] = [random_card]
            #print(f'Player Got the card " {random_subject} : {random_card} " from deck')
            deck[random_subject].remove(random_card)
            #check and remove if list in random_subject is empty
            if deck[random_subject] == []:
                del deck[random_subject]
            self.amount_of_cards = self.count_cards()
            return random_subject , random_card
        else:
            print(f'There is no cards left in deck')
            return None , None

    def deal_cards(self , amount_of_cards_to_deal , deck ):
        for i in range(min_cards_in_hand):
            self.add_card_from_deck(deck)
        self.amount_of_cards = self.count_cards()
        return self.cards

    def count_cards(self):
        return sum(list(map(lambda x: len(x) , self.cards.values() ) ) ) # total number of cards in hand
    
    def check_sets(self):
        return any([True for subject in self.cards if len(self.cards[subject]) == 4 ])
    
    def realize_sets(self):
        for subject in list(self.cards):
            if len(self.cards[subject]) == 4:
                self.sets += 1
                #print(f'Set was created for the subject "{subject}",number of set for player : {sets_player_1}')
                del self.cards[subject]
        self.amount_of_cards = self.count_cards()
    
    def completing_cards_from_deck(self, min_cards_in_hand , deck):
        if self.amount_of_cards < min_cards_in_hand:
            for i in range( min_cards_in_hand - self.amount_of_cards): #take cards from deck
                #print(f'amount_of_cards : {self.amount_of_cards} , i : {i}')
                self.add_card_from_deck(deck)
            self.amount_of_cards = self.count_cards() #update amount_of_cards after adding cards

    def add_card(self,subject_key ,card_value): 
        #subject key must exists in self.cards
        #try:
        self.cards[subject_key].append(card_value)
        self.amount_of_cards = self.count_cards()
        

    def remove_card(self , subject_key , card_value): #subject_key and card must exists in self.cards
        try:
            self.cards[subject_key].remove(card_value)
            if self.cards[subject_key] == []:
                del self.cards[subject_key]
            self.amount_of_cards = self.count_cards()
        except KeyError as ke:
            print(f'Did not found subject_key {subject_key} in cards')
        except ValueError as ve:
            print(f'Did not found card "{subject_key}:{card_value}"" in cards')
    
    def is_subject_key_exists(self,subject_key):
        return True if subject_key in self.cards else False

    def is_card_exists(self ,subject_key ,card_value ):
        if subject_key in self.cards and card_value in self.cards[subject_key]: #added condition 'subject_key in cards' for preventing key error 
            return True
        else:
            return False
            #print(f'The card { choose_card } exsits and added to list')
   

#**First Cards Dealt**

Create a dictionary of players that contain values of player class

In [108]:
players = {}
for x in range( 1, amount_of_players + 1 ):
    if not end_game:
        players[f'player_{x}'] = player()
        #deal cards for every player in players dict
        players[f'player_{x}'].deal_cards( min_cards_in_hand ,deck)

In [109]:
#check values
for player_num , player_val in list(players.items()) :
    print(player_num , player_val.cards)
#sorted(players)

player_1 {'subj4': ['subj_s2'], 'subj2': ['subjy3'], 'subj1': ['subjx3', 'subjx2']}
player_2 {'subj4': ['subj_s1', 'subj_s4'], 'subj3': ['subj_t4', 'subj_t3']}
player_3 {'subj3': ['subj_t2', 'subj_t1'], 'subj4': ['subj_s3'], 'subj1': ['subj_x1']}


For any player check if first dealt cards created a set

In [110]:
for player_num , player_val in players.items():
    while player_val.check_sets() and not end_game:
        player_val.realize_sets()
        player_val.completing_cards_from_deck(min_cards_in_hand , deck)
        print(f'{player_num} , values : {player_val}')

# **The Game** 
Players asking for cards

In [111]:
players_sets = [player.sets for player in players.values()]
#print(players_sets)
while not end_game and max(players_sets) < sets_to_win:
    #turns are changed by the for loop + and restarted once arrived to the while loop
    for player_num , player_val in players.items():
        guess = True
        while guess: #continue player turn if he had guess right ######
            print(f'######## {player_num} turn ######')
            player_val.completing_cards_from_deck(min_cards_in_hand , deck) #complete cards if needed
            while player_val.check_sets():
                player_val.realize_sets()
                player_val.completing_cards_from_deck(min_cards_in_hand , deck) #completing_cards
            #break if achieved sets_to_win
            players_sets = [player.sets for player in players.values()]
            print(f' {player_num} status : {player_val}')
            if max(players_sets) >= sets_to_win:
                end_game = True
                break #end of while "guess" loop
            #choose the opponent you want to ask for his cards
            opponent_num = input(f'choose the opponent you want to ask for his cards (number)')
            #choose_card #player can only choose a card with a subject appear in his hand
            subjects_options = player_val.cards.keys()
            choose_sub = input(f'please choose a subject from the list : {subjects_options}')
            is_sub_exists = players[f'player_{opponent_num}'].is_subject_key_exists(choose_sub)
            print(f'The subject selected" {choose_sub} " was { is_sub_exists } guess')
            
            #if subject exists choose card (sub theme)
            if is_sub_exists:
                cards_options = set(dict_[choose_sub]) - set(player_val.cards[choose_sub]) #apears in orginal dict_ but do not appear in player 1 cards
                choose_card = input(f'please choose a sub theme from the list : { cards_options }')  

            # check if choose_card exsits in opponent player cards
            if is_sub_exists and players[f'player_{opponent_num}'].is_card_exists(choose_sub , choose_card):
                print(f'The card { choose_card } exsits and added to list')
                player_val.add_card(choose_sub , choose_card)
                players[f'player_{opponent_num}'].remove_card(choose_sub , choose_card)
                guess = True
            else :
                guess = False
                print(f'The card { choose_card } not exsits in player_{opponent_num} cards')
                random_subject ,random_card = player_val.add_card_from_deck(deck)
                print(f' {player_num} got the card " {random_subject} : {random_card} " from deck') 
            #check if sets are made + complete cards from deck if needed
            while player_val.check_sets():
                player_val.realize_sets()
                player_val.completing_cards_from_deck(min_cards_in_hand , deck)
            #break if achieved sets_to_win
            players_sets = [player.sets for player in players.values()]
            print(f' {player_num} status : {player_val}')
            if max(players_sets) >= sets_to_win:
                end_game = True
                break #end of while "guess" loop
        if end_game:
            break #end of for loop on players

#check winner winner chicken dinner :)
Player_num_won = players_sets.index( max(players_sets) ) + 1
print(f'The winner is player number {Player_num_won} ')

######## player_1 turn ######
 player_1 status : (Cards : {'subj4': ['subj_s2'], 'subj2': ['subjy3'], 'subj1': ['subjx3', 'subjx2']} , sets: 0 , amount of cards in hand : 4 )
choose the opponent you want to ask for his cards (number)2
please choose a subject from the list : dict_keys(['subj4', 'subj2', 'subj1'])subj4
The subject selected" subj4 " was True guess
please choose a sub theme from the list : {'subj_s3', 'subj_s1', 'subj_s4'}subj_s3
The card subj_s3 not exsits in player_2 cards
 player_1 got the card " subj2 : subjy2 " from deck
 player_1 status : (Cards : {'subj4': ['subj_s2'], 'subj2': ['subjy3', 'subjy2'], 'subj1': ['subjx3', 'subjx2']} , sets: 0 , amount of cards in hand : 5 )
######## player_2 turn ######
 player_2 status : (Cards : {'subj4': ['subj_s1', 'subj_s4'], 'subj3': ['subj_t4', 'subj_t3']} , sets: 0 , amount of cards in hand : 4 )
choose the opponent you want to ask for his cards (number)1
please choose a subject from the list : dict_keys(['subj4', 'subj3'])subj